In [12]:
import pandas as pd
import numpy as np
import config as conf 

In [13]:
# Let's load the dataset and start the exploration.

df = pd.read_csv(conf.data_path,index_col=None)

df.head(3)

,Unnamed: 0,title,price,city,datetime,nb_rooms,nb_baths,surface_area,equipement,link
0,0,CMN-MA-1641 - Appartement à vendre à Val Fleurie,1 600 000 DH,Casablanca,"11/6/2024, 12:12:49 PM",3.0,1,132,"Ascenseur, Balcon, Parking, Terrasse",https://www.avito.ma/fr/val_fleuri/appartement...
1,1,Appartement à vendre 125 m² à Casablanca,9 900 DH,Casablanca,"11/6/2024, 12:12:26 PM",2.0,2,125,"Balcon, Climatisation, Cuisine Équipée, Parkin...",https://www.avito.ma/fr/autre_secteur/appartem...
2,2,CMN-MA-1787 - Appartement à vendre à Les Hôpitaux,2 300 000 DH,Casablanca,"11/6/2024, 12:10:48 PM",3.0,2,233,"Ascenseur, Balcon, Parking, Terrasse",https://www.avito.ma/fr/quartier_des_h%C3%B4pi...


In [14]:
# printing a full summary of our data.
df.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1760 entries, 0 to 1759
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    1760 non-null   int64  
 1   title         1400 non-null   object 
 2   price         1400 non-null   object 
 3   city          1400 non-null   object 
 4   datetime      1394 non-null   object 
 5   nb_rooms      1193 non-null   float64
 6   nb_baths      1181 non-null   object 
 7   surface_area  990 non-null    object 
 8   equipement    868 non-null    object 
 9   link          1760 non-null   object 
dtypes: float64(1), int64(1), object(8)
memory usage: 137.6+ KB


In [15]:
# Before starting the cleaning process let's normalize the column's naming.

df.rename(
    columns = {
        "equipement" : "extras"
        
    }
)



,Unnamed: 0,title,price,city,datetime,nb_rooms,nb_baths,surface_area,extras,link
0,0,CMN-MA-1641 - Appartement à vendre à Val Fleurie,1 600 000 DH,Casablanca,"11/6/2024, 12:12:49 PM",3.0,1,132,"Ascenseur, Balcon, Parking, Terrasse",https://www.avito.ma/fr/val_fleuri/appartement...
1,1,Appartement à vendre 125 m² à Casablanca,9 900 DH,Casablanca,"11/6/2024, 12:12:26 PM",2.0,2,125,"Balcon, Climatisation, Cuisine Équipée, Parkin...",https://www.avito.ma/fr/autre_secteur/appartem...
2,2,CMN-MA-1787 - Appartement à vendre à Les Hôpitaux,2 300 000 DH,Casablanca,"11/6/2024, 12:10:48 PM",3.0,2,233,"Ascenseur, Balcon, Parking, Terrasse",https://www.avito.ma/fr/quartier_des_h%C3%B4pi...
3,3,Appartement à vendre 3 chambres Salon à Casa,PRIX NON SPÉCIFIÉ,Casablanca,"11/6/2024, 12:10:57 PM",NaN,NaN,NaN,NaN,https://www.avito.ma/fr/ain_sebaa/appartements...
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://immoneuf.avito.ma/fr/unite/l53?utm_sou...
...,...,...,...,...,...,...,...,...,...,...
1755,1755,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://immoneuf.avito.ma/fr/unite/job?utm_sou...
1756,1756,CMN-AN-1524 - Appartement à vendre à Racine,3 300 000 DH,Casablanca,"9/17/2024, 10:44:45 AM",3.0,2,154,"Ascenseur, Parking",https://www.avito.ma/fr/racine/appartements/CM...
1757,1757,CMN-AC-1776 - Appartement à vendre à Californie,3 500 000 DH,Casablanca,"9/17/2024, 10:26:48 AM",3.0,2,170,"Ascenseur, Balcon, Parking, Terrasse",https://www.avito.ma/fr/californie/appartement...
1758,1758,Bel Appartement à vendre 117 m² à Casablanca,990 000 DH,Casablanca,"9/17/2024, 10:08:29 AM",2.0,2,117,"Ascenseur, Balcon, Parking, Sécurité",https://www.avito.ma/fr/ain_sebaa/appartements...


In [16]:
# Remove the first column (randomly generated while scraping)

df.drop('Unnamed: 0',axis = 1 ,inplace = True)

df.head(1)

,title,price,city,datetime,nb_rooms,nb_baths,surface_area,equipement,link
0,CMN-MA-1641 - Appartement à vendre à Val Fleurie,1 600 000 DH,Casablanca,"11/6/2024, 12:12:49 PM",3.0,1,132,"Ascenseur, Balcon, Parking, Terrasse",https://www.avito.ma/fr/val_fleuri/appartement...


In [17]:
df.shape

(1760, 9)

In [18]:
df.describe(include='all')

,title,price,city,datetime,nb_rooms,nb_baths,surface_area,equipement,link
count,1400,1400,1400,1394,1193.000000,1181,990,868,1760
unique,789,328,21,1392,NaN,12,203,260,1670
top,Appartement à vendre 50 m² à Casablanca,PRIX NON SPÉCIFIÉ,Casablanca,"10/4/2024, 12:22:09 PM",NaN,2,50,"Ascenseur, Balcon, Parking, Terrasse",https://immoneuf.avito.ma/fr/unite/opK?utm_sou...
freq,51,207,1336,2,NaN,560,34,73,5
mean,NaN,NaN,NaN,NaN,2.393965,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,0.746826,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,2.000000,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,2.000000,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,3.000000,NaN,NaN,NaN,NaN


seems like our data will need more cleaning than expected

In [19]:
# let's check how many missing values where are dealy with an where are they

df.isna().sum()


title           360
price           360
city            360
datetime        366
nb_rooms        567
nb_baths        579
surface_area    770
equipement      892
link              0
dtype: int64

Let's search for rows that are entierly empty

describe the df defect you had and why you chose this method

In [20]:
# Let's search for rows that are entierly empty
null_values = df.iloc[:,:8].isnull().all(axis = 1)

df = df[~null_values]

del null_values 


In [21]:
# checking if the amputation went well
df.isnull().sum()

title             0
price             0
city              0
datetime          6
nb_rooms        207
nb_baths        219
surface_area    410
equipement      532
link              0
dtype: int64

now we can start the data exploration and thorough cleaning

let's start by formating the data and adjusting the data types accordingly

In [ ]:
# The data is not correctly formated and types are all over the places.

# cleaning each field and adjusting its type

# Formatting and casting the price
df['price'] = df['price'].str.replace(r'\s+| DH', '', regex=True).str.replace('DH','') # for some reason this in the only one that works
df['price'] = pd.to_numeric(df['price'], errors='coerce')

In [ ]:
# doing the same for number of baths and rooms
df['nb_baths'] = pd.to_numeric(df['nb_baths'],errors = 'coerce')
df['nb_rooms'] = pd.to_numeric(df['nb_rooms'],errors = 'coerce')

In [57]:
# formating and casting the timestamps 

df['datetime'] = df['datetime'].str.replace(',',' ').str.replace(' PM','')

formatted_date = pd.to_datetime(df['datetime'], format='%d/%m/%Y %H:%M:%S', errors='coerce')




In [58]:
# df['nb_rooms'].describe()
formatted_date


0      2024-06-11 12:12:49
1      2024-06-11 12:12:26
2      2024-06-11 12:10:48
3      2024-06-11 12:10:57
5                      NaT
               ...        
1753                   NaT
1754                   NaT
1756                   NaT
1757                   NaT
1758                   NaT
Name: datetime, Length: 1400, dtype: datetime64[ns]

In [15]:
df['price'].unique()

array([nan])